In [19]:
# def fit_metrics_print(model, X_train, y_train, X_val, y_val):
#     from sklearn.metrics import accuracy_score, classification_report
#     model.fit(X_train, y_train)
    
#     pred_train = model.predict(X_train)
#     pred_val = model.predict(X_val)
#     print(model)
#     print()
#     print('train accuracy :', accuracy_score(y_train, pred_train))
#     print('validation accuracy :', accuracy_score(y_val, pred_val))
#     print('='*60)

In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [118]:
X = pd.read_csv('../data/ohe_edit_4.8.1(add_col).csv')
X.drop(columns='target', inplace=True)
y = pd.read_csv('../data/ohe_edit_4.8.1(add_col).csv')['target']
X_test = pd.read_csv('../data/prepcd_test.csv')

#### 우상님 코드용

In [119]:
# X.rename(columns={'company_size_<10':'company_size_~10'},inplace=True)

# X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, shuffle=True, random_state=1)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape

#### 내코드용 1차

In [120]:
# X = pd.read_csv('../data/prepcd_train.csv')
# X.drop(columns='target', inplace=True)
# y = pd.read_csv('../data/prepcd_train.csv')['target']
# X_test = pd.read_csv('../data/prepcd_test.csv')

#### 내코드용 2차

In [121]:
X = pd.read_csv('../data/prepcd_lbe_train.csv')
X.drop(columns='target', inplace=True)
y = pd.read_csv('../data/prepcd_lbe_train.csv')['target']
X_test = pd.read_csv('../data/prepcd_lbe_test.csv')

In [122]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, shuffle=True, random_state=1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((14368, 13), (4790, 13), (14368,), (4790,))

### Polynomial 컬럼 추가(Trial&Error) XGBoost모델 feature_importances_ 기준

### 중요도 컬럼뽑기 - 랜덤서치(nopoly,no oversampling) 한번 실행후

In [129]:
imprt_lst = pd.Series(be.feature_importances_, index=X_train.columns).sort_values(ascending=False)
to_poly_lst = list(imprt_lst[imprt_lst>imprt_lst.quantile(q=0.8)].index)
to_poly_lst

['city_development_index', 'company_size', 'education_level']

In [130]:
from sklearn.preprocessing import PolynomialFeatures

poly_f = PolynomialFeatures(degree=2, include_bias=False)
poly_f1 = PolynomialFeatures(degree=2, include_bias=False)

poly_train = poly_f.fit_transform(X.loc[:,X.columns.isin(to_poly_lst)])
poly_test = poly_f1.fit_transform(X_test.loc[:,X_test.columns.isin(to_poly_lst)])

X_train_poly_tomerge = pd.DataFrame(poly_train, columns=poly_f.get_feature_names())
X_test_poly_tomerge = pd.DataFrame(poly_test, columns=poly_f1.get_feature_names())

X_train_poly = pd.concat([X, X_train_poly_tomerge], axis=1)
X_test_poly = pd.concat([X_test, X_test_poly_tomerge], axis=1)

In [131]:
vars_poly = {f'x{i}':v for i,v in enumerate(to_poly_lst)}
vars_poly

{'x0': 'city_development_index', 'x1': 'company_size', 'x2': 'education_level'}

In [132]:
X_train_poly, X_val_poly, y_train_poly, y_val_poly = train_test_split(X_train_poly, y, stratify=y, shuffle=True, random_state=1)
X_train_poly.shape, X_val_poly.shape, y_train_poly.shape, y_val_poly.shape

((14368, 22), (4790, 22), (14368,), (4790,))

In [133]:
X_train_poly

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,...,training_hours,x0,x1,x2,x0^2,x0 x1,x0 x2,x1^2,x1 x2,x2^2
12751,0.147735,1.098572,-0.538003,0.203475,1.601582,-0.591065,-0.254614,-0.326356,-1.195199,-0.599850,...,2.108598,-0.538003,-0.254614,-0.599850,0.289448,0.136983,0.322721,0.064829,0.152730,0.359820
12061,-0.293801,0.561325,-1.655108,0.203475,-0.624383,-0.591065,-0.254614,-0.326356,0.278127,-0.249284,...,-1.037533,-1.655108,-0.254614,-0.249284,2.739382,0.421414,0.412592,0.064829,0.063471,0.062143
18919,-1.680622,0.137182,0.741001,0.203475,1.601582,1.701364,-0.254614,-0.326356,0.425459,1.152978,...,-0.221869,0.741001,-0.254614,1.152978,0.549082,-0.188669,0.854357,0.064829,-0.293565,1.329358
3215,0.836256,0.561325,-1.655108,0.203475,1.601582,-0.591065,-0.254614,-0.326356,-0.753201,0.802413,...,-0.804486,-1.655108,-0.254614,0.802413,2.739382,0.421414,-1.328079,0.064829,-0.204306,0.643866
5273,-1.340211,0.137182,0.741001,0.203475,-0.624383,-0.591065,-0.254614,-0.326356,-0.900534,-0.950415,...,2.308353,0.741001,-0.254614,-0.950415,0.549082,-0.188669,-0.704258,0.064829,0.241989,0.903289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,-0.100083,-1.106970,0.741001,-3.530470,1.601582,-0.591065,-0.254614,3.064143,-0.900534,1.152978,...,1.675797,0.741001,-0.254614,1.152978,0.549082,-0.188669,0.854357,0.064829,-0.293565,1.329358
8701,-1.574504,-1.106970,0.741001,0.203475,-0.624383,-0.591065,-0.254614,-0.326356,1.162122,-0.950415,...,-0.654670,0.741001,-0.254614,-0.950415,0.549082,-0.188669,-0.704258,0.064829,0.241989,0.903289
18883,0.096861,0.589601,0.571006,0.203475,-0.624383,-0.591065,-0.254614,-0.326356,1.604119,-0.249284,...,-0.687962,0.571006,-0.254614,-0.249284,0.326048,-0.145386,-0.142343,0.064829,0.063471,0.062143
17051,-0.807226,0.165458,-0.497528,0.203475,-0.624383,-0.591065,-0.254614,-0.326356,-0.311203,-1.651546,...,1.659151,-0.497528,-0.254614,-1.651546,0.247535,0.126678,0.821691,0.064829,0.420508,2.727605


### 오버샘플링

In [134]:
# !pip install imblearn

In [135]:
# from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

In [136]:
# oversample = SMOTE()
# sm = SMOTE(ratio='auto', kind='regular')
sm = SMOTE(random_state=0)
X_train_ovsp, y_train_ovsp = sm.fit_resample(X_train, list(y_train))

In [137]:
sm = SMOTE(random_state=0)
X_train_poly_ovsp, y_train_poly_ovsp = sm.fit_resample(X_train_poly, list(y_train_poly))

In [138]:
X_train_ovsp.shape, X_train.shape, X_train_poly_ovsp.shape, X_train_poly.shape

((21570, 13), (14368, 13), (21570, 22), (14368, 22))

- - -
# XGBoost_randomized

In [123]:
param = dict(
    n_estimators=[100, 200, 300, 400, 500],
    learning_rate=[0.001,0.005,0.01,0.05,0.1,0.5],
    max_depth=range(1, 7)
)

In [124]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200,
                   learning_rate=0.5,
                   max_depth=2,
                   random_state=1)
xgb.fit(X_train, y_train)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:57:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=12, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

#### nopoly-nooversampling

In [125]:
random_xgb = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb.fit(X_train, y_train)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:57:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [126]:
be = random_xgb.best_estimator_
pd.Series(be.feature_importances_, index=X_train.columns).sort_values(ascending=False)

city_development_index    0.308716
company_size              0.284353
education_level           0.098494
company_type              0.058500
relevent_experience       0.043561
last_new_job              0.032939
experience                0.030608
enrolled_university       0.029641
city                      0.027563
major_discipline          0.023135
enrollee_id               0.023065
training_hours            0.021272
gender                    0.018153
dtype: float32

In [127]:
pd.DataFrame(random_xgb.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,2.781230,0.010593,0.011403,0.003486,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.798330,0.786594,0.792023,0.792316,0.004795,1
5,10.922286,0.016012,0.014640,0.001380,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.798121,0.786385,0.790979,0.791829,0.004829,2
0,8.320877,0.061077,0.012857,0.000240,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.796033,0.789309,0.789726,0.791690,0.003076,3
1,9.447668,0.082990,0.012531,0.001892,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.792902,0.788474,0.789100,0.790158,0.001957,4
9,2.353213,0.076021,0.009454,0.000624,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.789562,0.784506,0.785341,0.786470,0.002213,5
2,5.950103,0.058845,0.029462,0.015458,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.788935,0.784297,0.783462,0.785565,0.002407,6
4,1.191626,0.031925,0.007821,0.000399,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.782672,0.780956,0.784089,0.782572,0.001281,7
7,7.341498,0.094911,0.012857,0.001961,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.782463,0.779703,0.784089,0.782085,0.001810,8
6,4.911180,0.062282,0.021814,0.006642,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.781420,0.781374,0.782209,0.781668,0.000383,9
3,3.542412,0.049038,0.016449,0.006062,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.783507,0.772813,0.780330,0.778883,0.004484,10


In [128]:
pred_train = random_xgb.predict(X_train)
pred_val = random_xgb.predict(X_val)

print('train accuracy :', accuracy_score(y_train, pred_train))
print('validation accuracy :', accuracy_score(y_val, pred_val))

train accuracy : 0.8167455456570156
validation accuracy : 0.7962421711899791


#### poly-nooversampling

In [139]:
random_xgb_poly = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_poly.fit(X_train_poly, y_train_poly)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [140]:
be_poly = random_xgb_poly.best_estimator_
pd.Series(be_poly.feature_importances_, index=X_train_poly.columns).sort_values(ascending=False)

company_size              0.296272
city_development_index    0.245560
x1^2                      0.112375
x0^2                      0.076204
x0 x2                     0.054392
relevent_experience       0.039928
company_type              0.024120
last_new_job              0.020131
experience                0.018034
enrolled_university       0.014673
x0 x1                     0.013214
x2^2                      0.012214
x1 x2                     0.012100
major_discipline          0.010724
city                      0.010476
education_level           0.010207
enrollee_id               0.010153
gender                    0.009709
training_hours            0.009514
x0                        0.000000
x1                        0.000000
x2                        0.000000
dtype: float32

In [141]:
pd.DataFrame(random_xgb_poly.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
5,16.709105,0.040552,0.019382,0.002437,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.801879,0.793067,0.794529,0.796492,0.003856,1
8,4.719409,0.082738,0.015953,0.004625,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.801044,0.789726,0.792859,0.794543,0.004771,2
1,9.764463,0.137971,0.019194,0.005765,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.798330,0.788265,0.791606,0.792733,0.004186,3
2,5.626392,0.040837,0.012050,0.000239,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.794990,0.789309,0.790353,0.791550,0.002469,4
0,8.510986,0.183664,0.027697,0.010086,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.794363,0.791188,0.786803,0.790785,0.003100,5
9,4.971837,0.047891,0.013833,0.001279,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.796660,0.786177,0.788891,0.790576,0.004442,6
7,12.079908,0.248717,0.021789,0.002296,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.795198,0.786803,0.788474,0.790158,0.003628,7
6,6.023155,0.072801,0.021966,0.005526,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.789562,0.780956,0.783671,0.784730,0.003592,8
4,0.995921,0.014192,0.011230,0.000682,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.782672,0.781374,0.784089,0.782712,0.001109,9
3,3.264971,0.018754,0.018544,0.002108,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.779541,0.777824,0.785341,0.780902,0.003216,10


In [142]:
pred_train_poly = random_xgb_poly.predict(X_train_poly)
pred_val_poly = random_xgb_poly.predict(X_val_poly)

print('train accuracy :', accuracy_score(y_train_poly, pred_train_poly))
print('validation accuracy :', accuracy_score(y_val_poly, pred_val_poly))

train accuracy : 0.8104816258351893
validation accuracy : 0.7960334029227557


#### nopoly-oversampling

In [143]:
random_xgb_ovsp = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_ovsp.fit(X_train_ovsp, y_train_ovsp)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:13:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [144]:
be_ovsp = random_xgb_ovsp.best_estimator_
pd.Series(be_ovsp.feature_importances_, index=X_train_ovsp.columns).sort_values(ascending=False)

company_type              0.233673
city_development_index    0.173910
company_size              0.161064
education_level           0.113087
last_new_job              0.093687
experience                0.067578
enrolled_university       0.052341
city                      0.036737
relevent_experience       0.028717
training_hours            0.015599
enrollee_id               0.010632
major_discipline          0.007751
gender                    0.005225
dtype: float32

In [145]:
pd.DataFrame(random_xgb_ovsp.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,3.186937,0.020430,0.014006,0.001017,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.710709,0.853268,0.867316,0.810431,0.070747,1
5,13.365206,0.110753,0.021640,0.002188,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.744924,0.826565,0.838526,0.803338,0.041593,2
0,10.595559,0.136370,0.019206,0.002401,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.619193,0.882476,0.897218,0.799629,0.127729,3
1,11.523465,0.093100,0.020833,0.004193,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.595549,0.885396,0.898748,0.793231,0.139888,4
2,7.453749,0.137309,0.015302,0.003087,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.640890,0.859110,0.871627,0.790542,0.105943,5
9,3.029065,0.018344,0.012375,0.001406,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.621836,0.869263,0.880111,0.790403,0.119277,6
6,5.739917,0.017151,0.015148,0.000797,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.757441,0.787761,0.798192,0.781131,0.017284,7
3,4.280045,0.090715,0.013995,0.001865,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.524618,0.898748,0.906815,0.776727,0.178299,8
7,9.277979,0.060324,0.021002,0.001594,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.751043,0.778860,0.786092,0.771998,0.015109,9
4,1.467173,0.008698,0.012701,0.001196,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.735327,0.743394,0.754381,0.744367,0.007809,10


In [146]:
pred_train_ovsp = random_xgb_ovsp.predict(X_train_ovsp)
pred_val_ovsp = random_xgb_ovsp.predict(X_val)

print('train accuracy :', accuracy_score(y_train_ovsp, pred_train_ovsp))
print('validation accuracy :', accuracy_score(y_val, pred_val_ovsp))

train accuracy : 0.8423273064441353
validation accuracy : 0.786847599164927


#### poly-oversampleling

In [147]:
random_xgb_poly_ovsp = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_poly_ovsp.fit(X_train_poly_ovsp, y_train_poly_ovsp)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:15:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [148]:
be_poly_ovsp = random_xgb_poly_ovsp.best_estimator_
pd.Series(be_poly_ovsp.feature_importances_, index=X_train_poly_ovsp.columns).sort_values(ascending=False)

x0^2                      0.260578
company_size              0.147483
company_type              0.116479
x1^2                      0.085367
last_new_job              0.071151
city_development_index    0.059949
enrolled_university       0.057367
x0 x2                     0.036990
relevent_experience       0.035466
experience                0.031004
x2^2                      0.025883
city                      0.016636
x1 x2                     0.014887
training_hours            0.010659
x0 x1                     0.007049
enrollee_id               0.006799
education_level           0.006708
major_discipline          0.005044
gender                    0.004500
x0                        0.000000
x1                        0.000000
x2                        0.000000
dtype: float32

In [149]:
pd.DataFrame(random_xgb_poly_ovsp.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,6.719591,0.261055,0.021478,0.005491,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.731154,0.859388,0.865090,0.818544,0.061838,1
5,21.474328,0.100275,0.020507,0.001428,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.760083,0.838248,0.843115,0.813815,0.038046,2
0,11.666732,0.612302,0.017099,0.001382,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.630181,0.886092,0.895132,0.803802,0.122824,3
9,5.787612,0.144927,0.017412,0.004627,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.642003,0.869263,0.876634,0.795967,0.108910,4
1,13.542342,0.236605,0.020677,0.002842,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.597079,0.889708,0.894854,0.793880,0.139175,5
2,8.451078,0.017842,0.019207,0.001891,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.650209,0.857858,0.868150,0.792072,0.100401,6
6,8.515727,0.190381,0.040748,0.010307,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.758554,0.784979,0.797914,0.780482,0.016380,7
3,5.246681,0.057999,0.019050,0.002108,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.527816,0.899444,0.908067,0.778442,0.177254,8
7,16.310473,0.141242,0.025415,0.004212,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.751599,0.769680,0.787900,0.769726,0.014820,9
4,1.603732,0.016060,0.012363,0.000460,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.739638,0.741586,0.753129,0.744784,0.005954,10


In [52]:
pred_train_poly_ovsp = random_xgb_poly_ovsp.predict(X_train_poly_ovsp)
pred_val_poly_ovsp = random_xgb_poly_ovsp.predict(X_val_poly)

print('train accuracy :', accuracy_score(y_train_poly_ovsp, pred_train_poly_ovsp))
print('validation accuracy :', accuracy_score(y_val, pred_val_poly_ovsp))

train accuracy : 0.8461288827074641
validation accuracy : 0.7870563674321504
